# Retrive info. from excel sheet and check with the names
## Packages will be used:
1. Marshmallow for validation for the names
## The SEQ. file names as following:
Each folder has a specific genotype, and in each folder, the file names are:
yy-nnnnn.xx A
- yy- : year e.g. 00- = 20(00)
- -nnnnn- : is the sequential number of package
- .xx is sequential number of the sample from that package (sampling type)
- A : String stand to the type, sometimes there 
The files doesn't have similar format gb, txt and FASTA. The script for change the format and validation of the format is 1-format.ipynb.

## FASTA file:
The FASTA file has the same names as the file, has more strings refering to sample type. It is not standardized as the file names, so similar problem as file names.

## The Excel sheet:
| SCount| Ein | EingangsDatum | EntnahmeDatum | ErkrBeginn | EkzBeginn | BLand | GesAmt | Material | Genotyp | WHO distinct SeqID |
|----------|----------|----------|----------|----------|----------|----------|----------|----------|----------|----------|
| 06-00025.01  | 70794 Filderstadt, Gemeinschaftspraxis  | 2/3/2006  | 2/1/2006  | 1/28/2006 | 1/31/2006 |Baden-Württemberg	| oral fluid | B3 | 2254 |

- SCount: patient ID (yy-nnnnn.xx) as in the file name
- Ein: Location of Sampling
- EingangsDatum: Date of entry
- EntnahmeDatum: Date of withdrawal 
- ErkrBeginn: Start of illness 
- EkzBeginn: Start of illness 
- BLand: Federal state  
- Total : ?

In general the naming of the files are not consistent across the folder, and doesn't match the excel sheet. As excel sheet have only yy-nnnnn.xx the serial number nothing added, and only 1 patient sample. While the folder has reptitive sequence for different samples and the name is not standardized across the file.


### The Goal:
The Goal is to have a SCount, file name and >FASTA name standardized across all.
1. Standardize the file naming across folder
    - yy-nnnnn.xx refers to only 1 sample type not several
    - Matching the files with excel sheet
    - The files, which are not in the excel will be put in a different folder and later I will check the sequence with other sampling in excel if it is identical
    - The sample type will be only in CSV file
2. Make a plot for the sampling type
3. Change the name inside the FASTA file to be as same as file name and SCount


Importation of Packages

In [1]:
#!pip install marshmallow

In [2]:
import pandas as pd
import glob
import os
import re
from marshmallow import Schema, fields, ValidationError
import shutil # to move files

A Table with Sampling Type and its abbreviations, as mentioned in excel sheet and folder

Could be replaced by dictionary

In [3]:
data_abb = {'Material-excel': ['oral fluid', 'Urin', 'Rachenabstrich', 'Rachenabstrich', 'ZKÜ-Urin', 'ZKÜ-Ra', 'ZKÜ-Ra'],
        'file-name': ['OF', 'U', 'R', 'Ra', 'ZKUE U', 'ZKUE R', 'ZKUE Ra' ]}

# Create DataFrame
df = pd.DataFrame(data_abb)

print(df)

   Material-excel file-name
0      oral fluid        OF
1            Urin         U
2  Rachenabstrich         R
3  Rachenabstrich        Ra
4        ZKÜ-Urin    ZKUE U
5          ZKÜ-Ra    ZKUE R
6          ZKÜ-Ra   ZKUE Ra


# 1. validation of the file names using Marshmallow

In [4]:
import os
from marshmallow import fields, ValidationError

class FileNameField(fields.Str):
    def _validate(self, value):
        # Validate file extension
        if not value.endswith(".fasta"):
            raise ValidationError(f"Invalid file extension for '{value}'. Expected '.fasta'.")

        try:
            # Remove the file extension and split the base name
            base_name = value[:-6]  # Remove '.fasta'
            yy, rest = base_name.split('-')
            nnnnn, xx = rest.split('.')

            # Ensure components are of the correct format
            if not (yy.isdigit() and len(yy) == 2):
                raise ValidationError(f"Invalid 'yy' value: '{yy}'. Must be a two-digit number.")
            if not (nnnnn.isdigit() and len(nnnnn) == 5):
                raise ValidationError(f"Invalid 'nnnnn' value: '{nnnnn}'. Must be a five-digit number.")
            if not (xx.isdigit() and len(xx) == 2):
                raise ValidationError(f"Invalid 'xx' value: '{xx}'. Must be a two-digit number.")

            # If needed, you can process or store these components
            self.yy = yy
            self.nnnnn = nnnnn
            self.xx = xx

        except ValueError:
            raise ValidationError(f"'{value}' does not match the required format 'yy.nnnnn-xx.fasta'.")

def validate_files_in_folder(folder_path):
    field = FileNameField()
    valid_files = []
    invalid_files = []

    for file_name in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file_name)):  # Ensure it's a file
            try:
                field._validate(file_name)  # Validate the file name
                valid_files.append(file_name)
            except ValidationError as e:
                invalid_files.append((file_name, str(e)))

    # Print results
    print("\nInvalid file names:")
    for file, error in invalid_files:
        print(f"{file}: {error}")

    print("\nValid file names:")
    for file in valid_files:
        print(file)

if __name__ == "__main__":
    folder_path = '/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/fasta'
    validate_files_in_folder(folder_path)



Invalid file names:
09-00209Urin.fasta: '09-00209Urin.fasta' does not match the required format 'yy.nnnnn-xx.fasta'.
13-00122Rsp.fasta: '13-00122Rsp.fasta' does not match the required format 'yy.nnnnn-xx.fasta'.
06-00036.fasta: '06-00036.fasta' does not match the required format 'yy.nnnnn-xx.fasta'.
13-00122Rsp copy.fasta: '13-00122Rsp copy.fasta' does not match the required format 'yy.nnnnn-xx.fasta'.
13-00125Rspcopy copy 2.fasta: '13-00125Rspcopy copy 2.fasta' does not match the required format 'yy.nnnnn-xx.fasta'.
09-00439.fasta: '09-00439.fasta' does not match the required format 'yy.nnnnn-xx.fasta'.
06-00058 OF.fasta: '06-00058 OF.fasta' does not match the required format 'yy.nnnnn-xx.fasta'.
13-00125Rspcopy copy.fasta: '13-00125Rspcopy copy.fasta' does not match the required format 'yy.nnnnn-xx.fasta'.
07-00235 copy.fasta: '07-00235 copy.fasta' does not match the required format 'yy.nnnnn-xx.fasta'.
06-00025.fasta: '06-00025.fasta' does not match the required format 'yy.nnnnn-xx

# 2. Make a dataframe with the file names
- Making a df with all file names
- split the file names to yy-nnnnn and the rest of it

In [5]:
# Specify the folder path
folder_path = '/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/fasta'

# Get the list of filenames in the folder
file_names = os.listdir(folder_path)

# Create a DataFrame
df_files = pd.DataFrame(file_names, columns=['FileName'])

df_files


,FileName
0,09-00209Urin.fasta
1,13-00122Rsp.fasta
2,06-00036.fasta
3,13-00122Rsp copy.fasta
4,13-00125Rspcopy copy 2.fasta
5,09-00439.fasta
6,06-00058 OF.fasta
7,13-00125Rspcopy copy.fasta
8,07-00235 copy.fasta
9,06-00025.fasta


In [6]:
# Split 'SCount' into two columns
df_files[['SCount_Prefix', 'SCount_Remainder']] = df_files['FileName'].str.extract(r'(\d{2}-\d{5})(.*)')
df_files['SCount_Remainder'] = df_files['SCount_Remainder'].str.replace('.fasta', '')

df_files.head()

,FileName,SCount_Prefix,SCount_Remainder
0,09-00209Urin.fasta,09-00209,Urin
1,13-00122Rsp.fasta,13-00122,Rsp
2,06-00036.fasta,06-00036,
3,13-00122Rsp copy.fasta,13-00122,Rsp copy
4,13-00125Rspcopy copy 2.fasta,13-00125,Rspcopy copy 2


# 3. Convert (excel file) to CSV

In [7]:
B3_excel = '/Users/MiladM-Dev/Documents/1PhD/Dataset_trials/B3_mimic/B3_example.csv'
B3xsl_df = pd.read_csv(B3_excel, encoding='utf-8', sep=';')
B3xsl_df.head()

,SCount,Ein,EingangsDatum,EntnahmeDatum,ErkrBeginn,EkzBeginn,BLand,GesAmt,Material,Genotyp,WHO distinct SeqID
0,06-00025.01,"70794 Filderstadt, Gemeinschaftspraxis",2/3/2006,2/1/2006,1/28/2006,1/31/2006,Baden-Württemberg,NaN,oral fluid,B3,2254
1,06-00036.02,"70771 Leinenfelden-Echterdingen, Dr. med. Iris...",2/10/2006,2/8/2006,2/4/2006,2/8/2006,Baden-Württemberg,NaN,oral fluid,B3,2254
2,06-00048.02,"70794 Filderstadt-Bonlanden, Gemeinnütziges Ge...",2/14/2006,2/6/2006,1/29/2006,2/5/2006,Baden-Württemberg,NaN,Rachenabstrich,B3,2254
3,06-00050.01,"70794 Filderstadt, Gemeinschaftspraxis",2/15/2006,2/14/2006,2/9/2006,2/11/2006,Baden-Württemberg,NaN,Rachenabstrich,B3,2254
4,06-00054.01,"70190 Stuttgart, Dr. med. Christoph Michels",2/20/2006,2/17/2006,2/15/2006,NaN,Baden-Württemberg,NaN,Rachenabstrich in Flüssigkeit,B3,2254


In [10]:
print(B3xsl_df.columns)

Index(['SCount', 'Ein', 'EingangsDatum', 'EntnahmeDatum', 'ErkrBeginn',
       'EkzBeginn', 'BLand', 'GesAmt', 'Material', 'Genotyp',
       'WHO distinct SeqID'],
      dtype='object')


In [11]:
B3_material = B3xsl_df[['SCount', 'Material']]
B3_material.head()

,SCount,Material
0,06-00025.01,oral fluid
1,06-00036.02,oral fluid
2,06-00048.02,Rachenabstrich
3,06-00050.01,Rachenabstrich
4,06-00054.01,Rachenabstrich in Flüssigkeit


In [12]:
# Split 'SCount' into two columns
B3_material[['SCount_Prefix', 'SCount_Suffix']] = B3_material['SCount'].str.extract(r'(\d{2}-\d{5})(.*)')

B3_material.head()


/var/folders/c5/srv5gzkd52l_9y41mpknm0t80000gp/T/ipykernel_95230/3409083381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B3_material[['SCount_Prefix', 'SCount_Suffix']] = B3_material['SCount'].str.extract(r'(\d{2}-\d{5})(.*)')
/var/folders/c5/srv5gzkd52l_9y41mpknm0t80000gp/T/ipykernel_95230/3409083381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B3_material[['SCount_Prefix', 'SCount_Suffix']] = B3_material['SCount'].str.extract(r'(\d{2}-\d{5})(.*)')


,SCount,Material,SCount_Prefix,SCount_Suffix
0,06-00025.01,oral fluid,06-00025,.01
1,06-00036.02,oral fluid,06-00036,.02
2,06-00048.02,Rachenabstrich,06-00048,.02
3,06-00050.01,Rachenabstrich,06-00050,.01
4,06-00054.01,Rachenabstrich in Flüssigkeit,06-00054,.01


### Joining 2 DF (excel and files)

In [14]:
# Merge DataFrames on 'SCount' and handle duplicates
df_merged = pd.merge(df_files, B3_material, on='SCount_Prefix', how='outer', suffixes=('_df_files', '_B3_material'))
df_merged['duplicates'] = df_merged.duplicated(subset=['SCount_Prefix'])
df_merged = df_merged[['SCount_Prefix', 'FileName', 'SCount', 
       'Material', 'SCount_Remainder', 'SCount_Suffix', 'duplicates']]
#print(df_merged.columns)
df_merged

,SCount_Prefix,FileName,SCount,Material,SCount_Remainder,SCount_Suffix,duplicates
0,06-00025,06-00025.fasta,06-00025.01,oral fluid,,.01,False
1,06-00036,06-00036.fasta,06-00036.02,oral fluid,,.02,False
2,06-00048,NaN,06-00048.02,Rachenabstrich,NaN,.02,False
3,06-00050,NaN,06-00050.01,Rachenabstrich,NaN,.01,False
4,06-00054,NaN,06-00054.01,Rachenabstrich in Flüssigkeit,NaN,.01,False
5,06-00057,NaN,06-00057.01,oral fluid,NaN,.01,False
6,06-00058,06-00058 OF.fasta,06-00058.01,oral fluid,OF,.01,False
7,06-00058,06-00058 Ra.fasta,06-00058.01,oral fluid,Ra,.01,True
8,07-00011,07-00011R (FTA).fasta,07-00011.01,cDNA,R (FTA),.01,False
9,07-00235,07-00235 copy.fasta,NaN,NaN,copy,NaN,False


In [ ]:
# Find the SCount_Prefix values that are repetitive (appear more than once)
repeated_prefixes = df_merged['SCount_Prefix'].value_counts()[df_merged['SCount_Prefix'].value_counts() > 1].index

# Filter the DataFrame to keep only the rows with repeated SCount_Prefix
df_repeated = df_merged[df_merged['SCount_Prefix'].isin(repeated_prefixes)]
df_repeated  = df_repeated.dropna(subset=['SCount'])
# Show the result
df_repeated


In [ ]:
import pandas as pd

# Sample DataFrame with NaN and non-NaN 'SCount'
data = {'FileName': ['06-00058 OF.fasta', '06-00058 Ra.fasta', '13-00122Rsp.fasta', '13-00122Rsp copy.fasta'],
        'SCount': ['06-00058.01', '06-00058.01', None, None],
        'SCount_Prefix': ['06-00058', '06-00058', '13-00122', '13-00122'],
        'Material': ['oral fluid', 'oral fluid', None, None],
        'SCount_Remainder': ['OF', 'Ra', 'Rsp', 'Rsp copy'],
        'SCount_Suffix': ['.01', '.01', None, None],
        'duplicates': [False, True, False, True]}

df = pd.DataFrame(data)

# Split the DataFrame into two: one with NaN in 'SCount' and one without
df_with_nan = df[df['SCount'].isna()]  # Rows where 'SCount' is NaN
df_without_nan = df[df['SCount'].notna()]  # Rows where 'SCount' is not NaN

# Show the results
print("DataFrame with NaN in 'SCount':")
print(df_with_nan)

print("\nDataFrame without NaN in 'SCount':")
df_without_nan
